In [ ]:
import pandas as pd
import numpy as np
import math

#读取文件
mobility_raw = pd.read_csv('mobility.csv',index_col=0)
InternetDelay_raw    = pd.read_csv('InternetDelay.csv',index_col=0)
load_raw     = pd.read_csv('load.csv',index_col=0)
cpuUtil_raw  = pd.read_csv('cpuUtil.csv',index_col=0)

# 得到cloudlet数量
cloudlet_numbers = mobility_raw.shape[1]

# 对comDelay和load进行同向化
# InternetDelay_txh = InternetDelay_raw.apply(lambda x: (np.max(x) - x)) 
# load_txh = load_raw.apply(lambda x: (np.max(x) - x)) 

# 合并成一个DataFrame
# data = pd.concat([mobility_raw,InternetDelay_txh,load_txh,cpuUtil_raw],axis=1)
data = pd.concat([mobility_raw,InternetDelay_raw,load_raw,cpuUtil_raw],axis=1)

# 对所有数据进行归一化
data_nor = data.apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x)))
# data_nor


In [ ]:
# 云模型处理

# def cloud_model(df):
#     Ex = df.mean()
#     En = np.sqrt(np.pi/2)  * np.mean( np.abs(df-Ex) )
#     He = np.sqrt(np.abs(df.mean() * df.mean() - df.std() * df.std()))
#     return pd.concat([Ex,En,He],axis=1)

# df.mad()  平均绝对离差
def cloud_model(df):
    Ex = df.mean()
    En = math.sqrt(math.pi / 2)  * df.mad()
    He = np.sqrt(np.abs(df.var() - En ** 2))
    return pd.concat([Ex,En,He],axis=1)
data_cloudModel = cloud_model(data_nor)
data_cloudModel.round(2)

In [ ]:
# 权重计算
ns_mobility = data_cloudModel.iloc[0:cloudlet_numbers]
ns_mobility_En = 0
for idx, row in ns_mobility.iterrows():
	ns_mobility_En += (row[0]+row[2]) * abs(2*row[1] - 1)
ns_mobility_En = 1 - ns_mobility_En / cloudlet_numbers

ns_InternetDelay = data_cloudModel.iloc[cloudlet_numbers:2*cloudlet_numbers]
ns_InternetDelay_En = 0
for idx, row in ns_InternetDelay.iterrows():
	ns_InternetDelay_En += (row[0]+row[2]) * abs(2*row[1] - 1)
ns_InternetDelay_En = 1 - ns_InternetDelay_En / cloudlet_numbers

ns_load = data_cloudModel.iloc[2*cloudlet_numbers:3*cloudlet_numbers]
ns_load_En = 0
for idx, row in ns_load.iterrows():
	ns_load_En += (row[0]+row[2]) * abs(2*row[1] - 1)
ns_load_En = 1 - ns_load_En / cloudlet_numbers

ns_cpuUtil = data_cloudModel.iloc[3*cloudlet_numbers:4*cloudlet_numbers]
ns_cpuUtil_En = 0
for idx, row in ns_cpuUtil.iterrows():
	ns_cpuUtil_En += (row[0]+row[2]) * abs(2*row[1] - 1)
ns_cpuUtil_En = 1 - ns_cpuUtil_En / cloudlet_numbers

# 中智熵
ns_En = [ns_mobility_En,ns_InternetDelay_En,ns_load_En,ns_cpuUtil_En]
# 属性权重
weight = []
for i in range(len(ns_En)):
	weight.append((1-ns_En[i]) / (4-sum(ns_En)))
print('属性权重为：',weight)

In [ ]:
# 中智聚合

# 中智数
truth = data_cloudModel[0]
indeterminacy = data_cloudModel[1]
falsity = data_cloudModel[2]

# 聚合之后的中智数
aggre_truth = []
aggre_indeterminacy = []
aggre_falsity = []

for i in range(cloudlet_numbers):
    tempT = 1
    tempI = 1
    tempF = 1
    # 四个属性
    for j in range(4):
        tempT *= (1 - truth[i+10*j]) ** weight[j]
        tempI *= indeterminacy[i+10*j] ** weight[j]
        tempF *= falsity[i+10*j] ** weight[j]
    aggre_truth.append((1-tempT))
    aggre_indeterminacy.append(tempI)
    aggre_falsity.append(tempF)

# 添加索引
# index = pd.Series(np.arange(0,cloudlet_numbers))
# index = index.astype(str)
# index = 'cloudlet'+index
neutrosophic = pd.DataFrame(np.vstack((aggre_truth,aggre_indeterminacy,aggre_falsity)).T)
neutrosophic.round(2)


In [ ]:
truth[10]

In [ ]:
# 得分函数
score = neutrosophic[0]+1-neutrosophic[1]+1-neutrosophic[2]
print(score.round(2))
print('最佳cloudlet为：',score.idxmax())

with open("optimal_cloudlet.txt","w") as f:
        f.write(str(score.idxmin()))  #这句话自带文件关闭功能，不需要再写f.close()
